Here I only test parts of the code before passing them on to the main notebook.

In [33]:
import time
import requests
from dotenv import load_dotenv
import os
import folium
import pandas as pd
import json

In [34]:
m = folium.Map(location=[20.9939879883004, -89.62853393602846], zoom_start=12, min_zoom=12, max_zoom=12)
delta = 0.01

initial_lat, initial_lng = 20.944451,-89.654108 


rectangles_viewports = []

for i in range(6):
    for j in range(6):
        low = [initial_lat+i*delta, initial_lng+j*delta]
        high = [initial_lat+(i+1)*delta, initial_lng+(j+1)*delta]
        folium.Rectangle(
            bounds = [low, high],
            tooltip = f'({i+1},{j+1})',
            fill = True
        ).add_to(m)
        
        rectangles_viewports.append((tuple(low), tuple(high)))
m

In [35]:
print(rectangles_viewports)

[((20.944451, -89.654108), (20.954451000000002, -89.64410799999999)), ((20.944451, -89.64410799999999), (20.954451000000002, -89.634108)), ((20.944451, -89.634108), (20.954451000000002, -89.62410799999999)), ((20.944451, -89.62410799999999), (20.954451000000002, -89.61410799999999)), ((20.944451, -89.61410799999999), (20.954451000000002, -89.604108)), ((20.944451, -89.604108), (20.954451000000002, -89.59410799999999)), ((20.954451000000002, -89.654108), (20.964451, -89.64410799999999)), ((20.954451000000002, -89.64410799999999), (20.964451, -89.634108)), ((20.954451000000002, -89.634108), (20.964451, -89.62410799999999)), ((20.954451000000002, -89.62410799999999), (20.964451, -89.61410799999999)), ((20.954451000000002, -89.61410799999999), (20.964451, -89.604108)), ((20.954451000000002, -89.604108), (20.964451, -89.59410799999999)), ((20.964451, -89.654108), (20.974451000000002, -89.64410799999999)), ((20.964451, -89.64410799999999), (20.974451000000002, -89.634108)), ((20.964451, -89.

In [36]:
load_dotenv()
API_KEY = os.getenv('API_KEY')

url = "https://places.googleapis.com/v1/places:searchText"
headers = {
    'Content-Type' : 'application/json',
    'X-Goog-Api-Key': API_KEY,
    'X-Goog-FieldMask': 'places.id,places.displayName'
}

In [37]:
raw_data = []
for low, high in rectangles_viewports:
    try: 
        payload = {
            'textQuery' : 'cafeteria',
            'includedType': 'cafe',
            'strictTypeFiltering':True,
            'locationRestriction' : {
                'rectangle':{
                    'low':{
                        'latitude' : low[0],
                        'longitude' : low[1]
                    },
                    'high':{
                        'latitude': high[0],
                        'longitude': high[1]
                        
                    }
                }
            }
        }
        
        response = requests.post(url = url, json=payload, headers = headers)
        response.raise_for_status()
        
        data = response.json()
        
        raw_data.extend(data.get('places', []))
        
    except requests.exceptions.RequestException as e:
        print(f'ERROR!!! --> {e}')
    
    time.sleep(0.5)


pd.DataFrame(raw_data).to_csv('places_id.csv')